<a href="https://colab.research.google.com/github/Nolanole/DS-Unit-2-Kaggle-Challenge/blob/master/DS_Sprint_Challenge_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science, Unit 2_
 
# Sprint Challenge: Predict Steph Curry's shots 🏀

For your Sprint Challenge, you'll use a dataset with all Steph Curry's NBA field goal attempts. (Regular season and playoff games, from October 28, 2009, through June 5, 2019.) 

You'll predict whether each shot was made, using information about the shot and the game. This is hard to predict! Try to get above 60% accuracy. The dataset was collected with the [nba_api](https://github.com/swar/nba_api) Python library.

In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [0]:
# Read data
import pandas as pd
url = 'https://drive.google.com/uc?export=download&id=1fL7KPyxgGYfQDsuJoBWHIWwCAf-HTFpX'
df = pd.read_csv(url)

# Check data shape
assert df.shape == (13958, 20)

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

You are permitted and encouraged to do as much data exploration as you want.

**1. Begin with baselines for classification.** Your target to predict is `shot_made_flag`. What is your baseline accuracy, if you guessed the majority class for every prediction?

**2. Hold out your test set.** Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

**3. Engineer new feature.** Engineer at least **1** new feature, from this list, or your own idea.
- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
- **Opponent**: Who is the other team playing the Golden State Warriors?
- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
- **Made previous shot**: Was Steph Curry's previous shot successful?

**4. Decide how to validate** your model. Choose one of the following options. Any of these options are good. You are not graded on which you choose.
- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
- **Train/validate/test split: random 80/20%** train/validate split.
- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

**5.** Use a scikit-learn **pipeline** to **encode categoricals** and fit a **Decision Tree** or **Random Forest** model.

**6.** Get your model's **validation accuracy.** (Multiple times if you try multiple iterations.) 

**7.** Get your model's **test accuracy.** (One time, at the end.)


**8.** Given a **confusion matrix** for a hypothetical binary classification model, **calculate accuracy, precision, and recall.**

### Stretch Goals
- Engineer 4+ new features total, either from the list above, or your own ideas.
- Make 2+ visualizations to explore relationships between features and target.
- Optimize 3+ hyperparameters by trying 10+ "candidates" (possible combinations of hyperparameters). You can use `RandomizedSearchCV` or do it manually.
- Get and plot your model's feature importances.



## 1. Begin with baselines for classification. 

>Your target to predict is `shot_made_flag`. What would your baseline accuracy be, if you guessed the majority class for every prediction?

In [0]:
import pandas_profiling

# df.profile_report()

In [12]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,scoremargin_before_shot
0,20900015,4,Stephen Curry,1,11,25,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,GSW,HOU,Regular Season,2.0
1,20900015,17,Stephen Curry,1,9,31,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,GSW,HOU,Regular Season,0.0
2,20900015,53,Stephen Curry,1,6,2,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,GSW,HOU,Regular Season,-4.0
3,20900015,141,Stephen Curry,2,9,49,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,GSW,HOU,Regular Season,-4.0
4,20900015,249,Stephen Curry,2,2,19,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,GSW,HOU,Regular Season,0.0


In [13]:
target = 'shot_made_flag'
df[target].value_counts(normalize=True)

0    0.527081
1    0.472919
Name: shot_made_flag, dtype: float64

## 2. Hold out your test set.

>Use the 2018-19 season to test. NBA seasons begin in October and end in June. You'll know you've split the data correctly when your test set has 1,709 observations.

In [14]:
df['game_date'] = pd.to_datetime(df['game_date'], infer_datetime_format=True)
df['game_date'].describe()

count                   13958
unique                    801
top       2013-05-06 00:00:00
freq                       35
first     2009-10-28 00:00:00
last      2019-06-05 00:00:00
Name: game_date, dtype: object

In [15]:
train = df[df['game_date'] < '10/1/2018']
test = df[df['game_date'] >= '10/1/2018']

train.shape, test.shape

((12249, 20), (1709, 20))

## 3. Engineer new feature.

>Engineer at least **1** new feature, from this list, or your own idea.
>
>- **Homecourt Advantage**: Is the home team (`htm`) the Golden State Warriors (`GSW`) ?
>- **Opponent**: Who is the other team playing the Golden State Warriors?
>- **Seconds remaining in the period**: Combine minutes remaining with seconds remaining, to get the total number of seconds remaining in the period.
>- **Seconds remaining in the game**: Combine period, and seconds remaining in the period, to get the total number of seconds remaining in the game. A basketball game has 4 periods, each 12 minutes long.
>- **Made previous shot**: Was Steph Curry's previous shot successful?

    

In [16]:
def feature_engineer(dataframe):
    df = dataframe.copy()
    df['HCA'] = (df['htm'] == 'GSW') * 1
    df['opponent'] = (df['htm'] + df['vtm']).str.strip('GSW')
    df['total_seconds_rem_period'] = df['minutes_remaining']*60 + df['seconds_remaining']
    df['total_seconds_rem_game'] = ((4-df['period']) * (12*60)) + df['total_seconds_rem_period']
    df['made_prev_shot'] = df[target].shift(1).fillna(-1)

    #drop home and visiting team, period, minutes, and seconds_remaining since its now duplicative information
    df = df.drop(columns=['htm', 'vtm', 'period', 'minutes_remaining', 'seconds_remaining'])
    return df

train = feature_engineer(train)
test = feature_engineer(test)

train.head()

,game_id,game_event_id,player_name,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,season_type,scoremargin_before_shot,HCA,opponent,total_seconds_rem_period,total_seconds_rem_game,made_prev_shot
0,20900015,4,Stephen Curry,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,26,99,249,0,2009-10-28,Regular Season,2.0,1,HOU,685,2845,-1.0
1,20900015,17,Stephen Curry,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-122,145,1,2009-10-28,Regular Season,0.0,1,HOU,571,2731,0.0
2,20900015,53,Stephen Curry,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,-60,129,0,2009-10-28,Regular Season,-4.0,1,HOU,362,2522,1.0
3,20900015,141,Stephen Curry,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,82,0,2009-10-28,Regular Season,-4.0,1,HOU,589,2029,0.0
4,20900015,249,Stephen Curry,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-68,148,0,2009-10-28,Regular Season,0.0,1,HOU,139,1579,0.0


## **4. Decide how to validate** your model. 

>Choose one of the following options. Any of these options are good. You are not graded on which you choose.
>
>- **Train/validate/test split: train on the 2009-10 season through 2016-17 season, validate with the 2017-18 season.** You'll know you've split the data correctly when your train set has 11,081 observations, and your validation set has 1,168 observations.
>- **Train/validate/test split: random 80/20%** train/validate split.
>- **Cross-validation** with independent test set. You may use any scikit-learn cross-validation method.

In [0]:
#Cross-validation in randomized search


## 5. Use a scikit-learn pipeline to encode categoricals and fit a Decision Tree or Random Forest model.

In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import category_encoders as ce 
from random import uniform, randint

# high_cardinality_cols = [col for col in train.select_dtypes(exclude='number').columns if train[col].nunique() > 32]
# low_cardinality_cols = set(train.select_dtypes(exclude='number').columns) - set(high_cardinality_cols)

# high_cardinality_cols.remove('game_date')

excluded_features = ['game_date', 'player_name', 'game_event_id']

target = 'shot_made_flag'
features = train.drop(columns=[target] + excluded_features).columns.tolist()

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12249, 16), (1709, 16), (12249,), (1709,))

In [19]:
pipe = make_pipeline(
    # make_column_transformer(
    #     (ce.OrdinalEncoder(), high_cardinality_cols),
    #     (ce.OneHotEncoder(use_cat_names=True), low_cardinality_cols),
    #     remainder='passthrough'
    # ),
    ce.OrdinalEncoder(),
    # ce.OneHotEncoder(use_cat_names=True),
    StandardScaler(),
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'randomforestclassifier__max_depth': [2,3,5,10,15,20,25,None],
    'randomforestclassifier__n_estimators': range(50, 500),
    'randomforestclassifier__max_features': [.2,.3,.4,.5,.6,.7,.8,.9,1],
    'randomforestclassifier__min_samples_leaf': [2,3,5,10],
    'randomforestclassifier__min_samples_split': [2,3,5,10],
}

search = RandomizedSearchCV(pipe, 
                            param_distributions= param_distributions,
                            n_iter = 100,
                            cv=8,
                            scoring='accuracy',
                            verbose=5,
                            return_train_score=True,
                            random_state=42,
                            n_jobs=-1)

search.fit(X_train, y_train)

Fitting 8 folds for each of 100 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  4.1min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 18.0min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 43.8min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed: 58.8min finished


RandomizedSearchCV(cv=8, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('standardscaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             wi

In [20]:
results = pd.DataFrame(search.cv_results_).sort_values('rank_test_score')
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__n_estimators,param_randomforestclassifier__min_samples_split,param_randomforestclassifier__min_samples_leaf,param_randomforestclassifier__max_features,param_randomforestclassifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,mean_train_score,std_train_score
78,5.022079,0.031548,0.088511,0.001725,255,10,3,0.5,5,"{'randomforestclassifier__n_estimators': 255, ...",0.697128,0.636423,0.669713,0.682768,0.620509,0.676471,0.685621,0.630719,0.662421,0.027030,1,0.662406,0.669777,0.666138,0.664085,0.672980,0.663215,0.663588,0.668346,0.666317,0.003498
39,3.548839,0.012577,0.069090,0.004365,180,2,10,0.5,5,"{'randomforestclassifier__n_estimators': 180, ...",0.697781,0.637076,0.669713,0.682115,0.618550,0.676471,0.686275,0.630719,0.662340,0.027454,2,0.662219,0.670057,0.665298,0.663245,0.673540,0.663495,0.662469,0.668439,0.666095,0.003878
1,3.449945,0.014856,0.082191,0.001895,217,5,3,0.4,5,"{'randomforestclassifier__n_estimators': 217, ...",0.697781,0.637076,0.669713,0.683420,0.619203,0.676471,0.683660,0.630719,0.662258,0.027175,3,0.661752,0.669031,0.664552,0.663712,0.672887,0.663401,0.663868,0.669652,0.666107,0.003649
42,2.450820,0.012841,0.061165,0.002312,153,2,10,0.4,5,"{'randomforestclassifier__n_estimators': 153, ...",0.696475,0.637728,0.670366,0.682115,0.619856,0.675817,0.679739,0.630719,0.661605,0.026247,4,0.660539,0.669590,0.665485,0.663525,0.672980,0.662748,0.664521,0.669465,0.666107,0.003913
15,5.583204,0.015723,0.076149,0.003516,203,3,3,0.8,5,"{'randomforestclassifier__n_estimators': 203, ...",0.697781,0.637076,0.666449,0.682115,0.615284,0.675163,0.686275,0.630719,0.661360,0.027942,5,0.662965,0.670523,0.666791,0.665765,0.674193,0.662935,0.664987,0.669092,0.667157,0.003653


## 6.Get your model's validation accuracy

> (Multiple times if you try multiple iterations.)

In [21]:
search.best_score_

0.6624214221569108

## 7. Get your model's test accuracy

> (One time, at the end.)

In [22]:
from sklearn.metrics import accuracy_score

best_model = search.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.6214160327677004

## 8. Given a confusion matrix, calculate accuracy, precision, and recall.

Imagine this is the confusion matrix for a binary classification model. Use the confusion matrix to calculate the model's accuracy, precision, and recall.

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

### Calculate accuracy 

In [0]:
total_predictions = 85 + 58 + 8 + 36
true_positives = 36
true_negatives = 85
false_positives = 58
false_negatives = 8
accuracy = (true_positives + true_negatives)/total_predictions
accuracy

### Calculate precision

In [0]:
precision = true_positives / (true_positives + false_positives)
precision

### Calculate recall

In [0]:
recall = true_positives / (true_positives + false_negatives)
recall